# Parse data

处理数据：
1. 分词
2. 

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt

DATA_DIR = '../data'
# DATA_FILE = 'mirror.csv'
DATA_FILE = 'xss.csv'

data_file = os.path.join(DATA_DIR, DATA_FILE)

In [2]:
data = pd.read_csv(data_file)
data.head()

,Datesubmitted,Datepublished,Status,Author,Domain,Category,Pagerank,URL
0,18/08/2014,13/03/2015,UNFIXED,SquirrelBuddha,webcenters.netscape.compuserve.com,XSS,70880,http://webcenters.netscape.compuserve.com/weat...
1,05/01/2014,13/03/2015,FIXED,puritys,store.samsung.com,XSS,340,"http://store.samsung.com/uk/camera/smart-n2x"">..."
2,15/11/2013,13/03/2015,FIXED,ArianaGrande,auth.dhs.gov,XSS,4057,https://auth.dhs.gov/oam/hsinlogin/?authn_try_...
3,06/10/2014,13/03/2015,UNFIXED,MrCyph3r,www.titivillus.it,XSS,11882046,http://www.titivillus.it/periodico.php?id=15%3...
4,09/08/2012,13/03/2015,FIXED,FabianCuchietti,www.brazzers.com,XSS,1755,http://www.brazzers.com/series/view/episodes/%...


取出5条url分析

In [46]:
urls = data['URL'][0:5].tolist()
for url in urls:
    print(url, end='\n{}\n'.format('-'*120))

http://webcenters.netscape.compuserve.com/weather/find.jsp?f="><script>alert(1)</script>
------------------------------------------------------------------------------------------------------------------------
http://store.samsung.com/uk/camera/smart-n2x"><script>alert(document.cookie)</script></nx1010-smart-camera/p/ED-LF52PL
------------------------------------------------------------------------------------------------------------------------
https://auth.dhs.gov/oam/hsinlogin/?authn_try_count=0&contextType=external&username=string&ssoCookie:max-age=3600&contextValue=%2Foam&password=sercure_string&challenge_url=%2Fhsinlogin&ssoCookie=max-age%3D3600&request_id=%27%22--%3E%3Cmarquee%3E%3Ccenter%3E%3C%2Fimg%3E%3Cbr%3E+%3Cbr%3E%3Cscript%3Ealert%28document.cookie%29;%3C/script%3E
------------------------------------------------------------------------------------------------------------------------
http://www.titivillus.it/periodico.php?id=15%3Cscript%3Ealert%28document.cookie%29%3C/scri

去除domain和路径信息

In [54]:
import re
def remove_domain(url):
    res = re.findall(r'http[s]?://([a-z0-9\.]+/)+(.+)', url)
    if res:
        return res[0][1]
    
    return []

uris = []
for url in urls:
    tmp_uri = remove_domain(url)
    uris.append(tmp_uri)
    print('{}\n=> {}'.format(url, tmp_uri), end='\n{}\n'.format('-'*120))

http://webcenters.netscape.compuserve.com/weather/find.jsp?f="><script>alert(1)</script>
=> find.jsp?f="><script>alert(1)</script>
------------------------------------------------------------------------------------------------------------------------
http://store.samsung.com/uk/camera/smart-n2x"><script>alert(document.cookie)</script></nx1010-smart-camera/p/ED-LF52PL
=> smart-n2x"><script>alert(document.cookie)</script></nx1010-smart-camera/p/ED-LF52PL
------------------------------------------------------------------------------------------------------------------------
https://auth.dhs.gov/oam/hsinlogin/?authn_try_count=0&contextType=external&username=string&ssoCookie:max-age=3600&contextValue=%2Foam&password=sercure_string&challenge_url=%2Fhsinlogin&ssoCookie=max-age%3D3600&request_id=%27%22--%3E%3Cmarquee%3E%3Ccenter%3E%3C%2Fimg%3E%3Cbr%3E+%3Cbr%3E%3Cscript%3Ealert%28document.cookie%29;%3C/script%3E
=> ?authn_try_count=0&contextType=external&username=string&ssoCookie:max-age=3600&

将去除后的信息进行分词

In [81]:
from urllib import parse
def split_word(sentence):
    """根据标点符号对句子进行分词
    """
    # 返回query, 如果query为空，则返回path
    # 去掉资源定位
    parse_res = parse.urlparse(sentence)
    res = parse_res.query if parse_res.query else parse_res.path
    # res = parse.parse_qs(res)
    
    # 使用url encode后的符号作为分隔符(%xx)
    sentences = [item.lower() for item in re.split(r'(%\w{2})', res) if item]
    
    # 提取
    words = []
    for st in sentences:
        if not re.match(r'%\w{2}', st):
            for re_res in re.split(r'(\W)', st):
                if re_res:
                    words.append(re_res)
        else:
            words.append(st)
    return words

for uri in uris:
    tmp_words = split_word(uri)
    print(tmp_words)

['f', '=', '"', '>', '<', 'script', '>', 'alert', '(', '1', ')', '<', '/', 'script', '>']
['smart', '-', 'n2x', '"', '>', '<', 'script', '>', 'alert', '(', 'document', '.', 'cookie', ')', '<', '/', 'script', '>', '<', '/', 'nx1010', '-', 'smart', '-', 'camera', '/', 'p', '/', 'ed', '-', 'lf52pl']
['authn_try_count', '=', '0', '&', 'contexttype', '=', 'external', '&', 'username', '=', 'string', '&', 'ssocookie', ':', 'max', '-', 'age', '=', '3600', '&', 'contextvalue', '=', '%2f', 'oam', '&', 'password', '=', 'sercure_string', '&', 'challenge_url', '=', '%2f', 'hsinlogin', '&', 'ssocookie', '=', 'max', '-', 'age', '%3d', '3600', '&', 'request_id', '=', '%27', '%22', '-', '-', '%3e', '%3c', 'marquee', '%3e', '%3c', 'center', '%3e', '%3c', '%2f', 'img', '%3e', '%3c', 'br', '%3e', '+', '%3c', 'br', '%3e', '%3c', 'script', '%3e', 'alert', '%28', 'document', '.', 'cookie', '%29', ';', '%3c', '/', 'script', '%3e']
['id', '=', '15', '%3c', 'script', '%3e', 'alert', '%28', 'document', '.', 'coo

词频统计，将单词编码为数字